In [1]:
import sys
sys.path.append('../')
from src import load_dataset

import spacy
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/charles/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
X, y = load_dataset()
nlp = spacy.load("fr_core_news_md")

In [3]:
def my_analyzer(X):
    """
    Use spacy to do everything
    """
    doc = nlp(X)
    l = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        else:
            if token.like_num:
                l.append("NUMBER")
            else:
                l.append(token.lemma_)
    return l
vectorizer = CountVectorizer(
    analyzer=my_analyzer,
)
# X_transformed = vectorizer.fit_transform(X)
X_transformed = np.load('./X_transformed_spacy_stop_punct_likeNum_lemma')

100%|██████████| 57413/57413 [09:21<00:00, 102.32it/s]


In [6]:
# import numpy as np
# np.save('./X_transformed_spacy_stop_punct_likeNum_lemma', X_transformed)

In [7]:
from sklearn.model_selection import train_test_split

# with a seed to enhence reproducibility
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.4, random_state=0)

from asyncore import write
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from src import write_classif_report

#Naïve Bayes
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

#Logistic Regression
lr_clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=100,n_jobs=-1)
lr_clf.fit(X_train, y_train)

#Linear SVM
svm_clf = LinearSVC(random_state=0, tol=1e-5)
svm_clf.fit(X_train, y_train)

pred_nb = nb_clf.predict(X_test)
pred_lr = lr_clf.predict(X_test)
pred_svm = svm_clf.predict(X_test)


print(f"Naïve Bayes accuracy: \n {classification_report(y_test, pred_nb)}")
print(f"Logistic Regression accuracy: \n {classification_report(y_test, pred_lr)}")
print(f"SVM accuracy: \n {classification_report(y_test, pred_svm)}")

/tmp/ipykernel_29055/191911167.py:6: DeprecationWarning: The asyncore module is deprecated and will be removed in Python 3.12. The recommended replacement is asyncio
  from asyncore import write
/home/charles/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Naïve Bayes accuracy: 
               precision    recall  f1-score   support

          -1       0.59      0.34      0.43      2998
           1       0.91      0.97      0.93     19968

    accuracy                           0.88     22966
   macro avg       0.75      0.65      0.68     22966
weighted avg       0.87      0.88      0.87     22966

Logistic Regression accuracy: 
               precision    recall  f1-score   support

          -1       0.69      0.32      0.44      2998
           1       0.91      0.98      0.94     19968

    accuracy                           0.89     22966
   macro avg       0.80      0.65      0.69     22966
weighted avg       0.88      0.89      0.88     22966

SVM accuracy: 
               precision    recall  f1-score   support

          -1       0.57      0.39      0.46      2998
           1       0.91      0.96      0.93     19968

    accuracy                           0.88     22966
   macro avg       0.74      0.67      0.70     22966
we

/home/charles/.local/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Score moins bon que de base pour dix fois plus de temps de train